In [1]:
import time, sys
import random as rd
import numpy as np
import itertools
from pickle import dump, load
from tqdm import tqdm_notebook as tqdm

from application import App
from actions import Actions
from keras.preprocessing import sequence

DIM = 3
available_actions = Actions.ACTIONS

Using TensorFlow backend.


In [2]:
app = App(DIM)

temp = ['<start>', '<end>']
for action in available_actions:
    temp.append(action)
            
app.restApp()
print('finish')

action2int = {word:i for i, word in enumerate(temp)}
int2action = {value:key for key, value in action2int.items()}
del temp
                              
                    
print('finish!')
app.quit(0.5)

finish
finish!


In [3]:
app = App(DIM)


actionsStatesStorage = {}
count = 0
for _ in tqdm(range(100000)):
    # make some actions
    iteration = rd.randint(1,20)
    app.manyActionsState(iteration)
    merged = list(itertools.chain(*app.myMasterAsInt.tolist()))
    merged = list(itertools.chain(*merged))
    key = merged.__str__()
    actionsList = ['<start>'] + app.reversedOppositeActions + ['<end>']
    
    if len(actionsList) > 2:
        if key not in actionsStatesStorage:
            actionsStatesStorage[key] = []
            actionsStatesStorage[key].append(actionsList)
        else:
            if actionsList not in actionsStatesStorage[key]:
                actionsStatesStorage[key].append(actionsList)
        
    if app.num_solved_sides() == 2:
        count += 1
        
    app.restApp()

print(count)
trainSize = int(len(actionsStatesStorage) * 0.9)
testSize = len(actionsStatesStorage) - trainSize

print(trainSize, testSize)

trainActionsStates = dict( list(actionsStatesStorage.items())[:trainSize] )    
testActionsStates = dict(list(actionsStatesStorage.items())[trainSize:])
    
with open("trainActionsStates.pkl", "wb") as file:
    dump(trainActionsStates, file)
with open("testActionsStates.pkl", "wb") as file:
    dump(testActionsStates, file)                               
                              
                               
                               
                               
print(len(trainActionsStates), len(testActionsStates))



del actionsStatesStorage
print('finish!')
app.quit()


6878
75266 8363
75266 8363
finish!


In [ ]:
with open("trainActionsStates.pkl", "rb") as file:
    trainActionsStates = load(file)
    
caption_length = [len(sentence) for liste in trainActionsStates.values() for sentence in liste]
max_length = max(caption_length)
vocab_size  = 20
print(max_length) # maximum lenght of a caption.


train_keys = []
train_values = []
for key, value in tqdm(trainActionsStates.items()):
    
    myKey = [int(char) for char in key[1:-1].split(', ')]
    myValue = value
    train_keys.append(myKey)
    train_values.append(myValue)
    


with open("testActionsStates.pkl", "rb") as file:
    testActionsStates = load(file)    

test_keys = []
test_values = []
for key, value in tqdm(testActionsStates.items()):
    
    myKey = [int(char) for char in key[1:-1].split(', ')]
    myValue = value
    test_keys.append(myKey)
    test_values.append(myValue)


In [ ]:
# data generator, intended to be used in a call to model.fit_generator()
import sys

def data_process(keys, values, batch_size):
    partial_captions = []
    next_words = []
    images = []
    total_count = 0
    while 1:
        
        for key, liste in zip(keys, values):
            current_image = key
            
            for sentense in liste:
                for i in range(len(sentense)-1):
                    total_count += 1
                    partial = [action2int[txt] for txt in sentense[:i+1]]
                    next = np.zeros(vocab_size)
                    next[ action2int[sentense[i+1]] ] = 1

                    partial_captions.append(partial)
                    next_words.append(next)
                    images.append(current_image)

                    if total_count>=batch_size:
                        next_words = np.asarray(next_words)
                        images = np.asarray(images)
                        partial_captions = sequence.pad_sequences(partial_captions, maxlen=max_length, padding='post')
                        total_count = 0

                        yield [[images, partial_captions], next_words]
                        #print(images.shape, partial_captions.shape, next_words.shape)
                        partial_captions = []
                        next_words = []
                        images = []


            
            #sys.exit()
#data_process(train_keys, train_values, 4)

In [ ]:
from keras.layers import Conv2D,RepeatVector, Activation, LSTM, Embedding, Dense,Dropout, BatchNormalization, Reshape, Flatten
from keras.layers.merge import add
from keras.models import Model
from keras import Input
from keras.layers.wrappers import Bidirectional


EMBEDDING_DIM = 300



def createModel():
    inputs1 = Input(shape=(54,))
    inner = Reshape((18,3,1))(inputs1)
    inner = Conv2D(256, kernel_size=(3, 3),activation='relu')(inner)
    inner = Flatten()(inner)
    inner = Dense(128, activation='relu')(inner)
    fe = RepeatVector(max_length)(inner)


    inputs2 = Input(shape=(max_length,))
    inner = Embedding(vocab_size,EMBEDDING_DIM , input_length=max_length)(inputs2)
    inner = Bidirectional(LSTM(256,return_sequences=True))(inner)
    inner = Dropout(0.5)(inner)
    inner = BatchNormalization()(inner)
    se = Dense(128)(inner)

    decoder1 = add([fe, se])
    inner = Dropout(0.5)(decoder1)
    inner = BatchNormalization()(inner)
    inner = Bidirectional(LSTM(1000,return_sequences=False))(inner)
    inner = Dense(vocab_size)(inner)
    outputs = Activation('softmax')(inner)
                        
                        
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)    
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    
    print ("Model created!")
    return model


model = createModel()
model.summary()

In [ ]:
epochs = 300
batch_size = 16
steps_per_epoch = len(train_keys)/batch_size

train_generator = data_process(train_keys, train_values, batch_size)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=steps_per_epoch, 
                              epochs=epochs, 
                              verbose=1, 
                              callbacks=None)
model.save('myModel.h5')

import matplotlib.pyplot as plt
%matplotlib inline



acc = history.history['acc']

loss = history.history['loss']


x = [i for i in range(1, epochs +1)]
plt.figure(figsize=(20,5))

plt.subplot(1,2,1)
plt.plot(x, acc, 'b', label='Training acc')
plt.title('Training accuracy')
plt.legend()


plt.subplot(1,2,2)
plt.plot(x, loss, 'b', label='Training loss')
plt.title('Training loss')
plt.legend()